In [1]:
pip install langdetect googletrans==4.0.0-rc1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 8.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached idna-2.10-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached h2-3.2.0-py2.py3-none-any.whl.metadata (32 kB)
  Using cached hyperframe-5.2.0-py2.py3-none-any.whl.metadata (7.2 kB)
  Using cached hpack-3.0.0-py2.py3-none-any.whl.metadata (7.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.8 MB/s eta 0:00:00
Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
Using cached h2-3.2.0-py2.py3-none-any.whl (65 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.5 MB/s eta 0:00:00
Using cached hpack-3.0.0-py2.py3-none-any.whl (38 kB)
Using 

In [12]:
pip install --upgrade scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langdetect import detect
from googletrans import Translator
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import joblib

In [2]:
df = pd.read_csv("medquad_qa.csv")
vectorizer = joblib.load("saved_model/vectorizer.pkl")
X = joblib.load("saved_model/X.pkl")
translator = Translator()

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.5.1 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.5.1 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
def multilingual_chatbot(user_input):
    # Detect language
    detected_lang = detect(user_input)

    if detected_lang != 'en':
        translated_input = translator.translate(user_input, src=detected_lang, dest='en').text
    else:
        translated_input = user_input

    user_vec = vectorizer.transform([translated_input])
    similarity = cosine_similarity(user_vec, X).flatten()
    best_idx = similarity.argmax()

    if similarity[best_idx] < 0.2:
    return "I'm sorry, I couldn't find a good answer for that.", detect(user_input)

    best_answer = df.iloc[best_idx]["answer"]

    if detected_lang != 'en':
        translated_output = translator.translate(best_answer, src='en', dest=detected_lang).text
    else:
        translated_output = best_answer

    return translated_output, detected_lang


In [7]:
while True:
    user_q = input("You: ")
    if user_q.lower() in ['exit', 'quit']:
        print("Bot: Thank you! 🙏")
        break

    response, lang = multilingual_chatbot(user_q)
    print(f"Bot ({lang.upper()}): {response}\n")

You:  अस्थमा क्या है?


Bot (HI): निमोनिया एक संक्रमण है जो एक या दोनों फेफड़ों में हवा की थैली को भड़काता है।



KeyboardInterrupt: Interrupted by user